In [1]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split('_')[0]

dirs = [
    '/home/chesetti/Repos/LearnedJoinDiskExp/sponge/merge_ssd',
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.num_output_keys', 'result.outer_disk_fetch',
       'result.outer_disk_fetch_size', 'result.outer_total_bytes_fetched',
       'spec.algo', 'spec.algo_name', 'spec.check_checksum', 'spec.common_key',
       'spec.index.epsilon', 'spec.index.leaf_size_in_pages',
       'spec.index.search', 'spec.index.type', 'spec.inner_table',
       'spec.key_size', 'spec.key_type', 'spec.load_sstable_in_mem',
       'spec.name', 'spec.num_threads', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk', 'iostat.bytes_read',
       'iostat.bytes_wrtn'],
      dtype='object')


,command,dataset,result.checksum,result.duration_ns,result.duration_sec,result.inner_disk_fetch,result.inner_disk_fetch_size,result.inner_total_bytes_fetched,result.num_output_keys,result.outer_disk_fetch,...,spec.key_type,spec.load_sstable_in_mem,spec.name,spec.num_threads,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk,iostat.bytes_read,iostat.bytes_wrtn
0,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,24971741285,24.971741,781256,4096,3200021488,399999998,781256,...,uint64,False,learned_merge_btree2561run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input1,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,6264496000,3687984000
1,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8214785197,8.214785,781255,4096,3200017088,219031081,74344,...,uint64,False,learned_merge_flatpgm25610run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3428224000,840044000
2,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8424467228,8.424467,781255,4096,3200017088,219031081,74344,...,uint64,False,learned_merge_btree25610run_1,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3436252000,867616000
3,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,7101072165,7.101072,781255,4096,3200016736,200199897,784,...,uint64,False,learned_merge_btree2561000run_1,8,sponge/merge_ssd/fb_threads=8/inputs/input1000,sponge/merge_ssd/fb_threads=8/outputs/learned_...,8,True,3141980000,603352000
4,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,8355656122,8.355656,781255,4096,3200017088,219031081,74344,...,uint64,False,standard_merge10run_2,8,sponge/merge_ssd/fb_threads=8/inputs/input10,sponge/merge_ssd/fb_threads=8/outputs/standard...,8,True,3428260000,897104000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,17388020872,17.388021,781250,4096,3199999984,219032474,74346,...,uint64,False,learned_merge_btree25610run_0,1,sponge/merge_ssd/fb_threads=1/inputs/input10,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,3436328000,3138264000
176,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,58861384880,58.861385,781250,4096,3199999984,399999998,781250,...,uint64,False,learned_merge_btree2561run_1,1,sponge/merge_ssd/fb_threads=1/inputs/input1,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,6264204000,3674140000
177,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,11868669099,11.868669,781250,4096,3199999984,201989891,7774,...,uint64,False,standard_merge100run_0,1,sponge/merge_ssd/fb_threads=1/inputs/input100,sponge/merge_ssd/fb_threads=1/outputs/standard...,8,True,3162072000,680348000
178,sponge/merge_ssd/build/benchmark_runner sponge...,fb,0,35775961271,35.775961,781250,4096,3199999984,399999998,781250,...,uint64,False,learned_merge_flatpgm2561run_1,1,sponge/merge_ssd/fb_threads=1/inputs/input1,sponge/merge_ssd/fb_threads=1/outputs/learned_...,8,True,6257216000,3674700000


In [2]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["inner_disk_fetch"] = df["result.inner_disk_fetch"]
df['result_num_keys'] = df['result.num_output_keys']
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["epsilon"] = df["spec.index.epsilon"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['disk_read_gb'] = df['iostat.bytes_read']/1e9
df['disk_wrtn_gb'] = df['iostat.bytes_wrtn']/1e9
#df['cdisk_read_gb'] = df['cgroup_iostat.bytes_read']/1e9
#df['cdisk_wrtn_gb'] = df['cgroup_iostat.bytes_wrtn']/1e9
df['checksum'] = df['result.checksum']
df['threads'] = df['spec.num_threads']

df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌──────────────────────┬─────────┬─────────────────┬────────────────────┬───┬──────────────┬──────────────┬──────────┐
│       command        │ dataset │ result.checksum │ result.duration_ns │ … │ disk_read_gb │ disk_wrtn_gb │ checksum │
│       varchar        │ varchar │      int64      │       int64        │   │    double    │    double    │  int64   │
├──────────────────────┼─────────┼─────────────────┼────────────────────┼───┼──────────────┼──────────────┼──────────┤
│ sponge/merge_ssd/b…  │ fb      │               0 │        24971741285 │ … │     6.264496 │     3.687984 │        0 │
│ sponge/merge_ssd/b…  │ fb      │               0 │         8214785197 │ … │     3.428224 │     0.840044 │        0 │
│ sponge/merge_ssd/b…  │ fb      │               0 │         8424467228 │ … │     3.436252 │     0.867616 │        0 │
│ sponge/merge_ssd/b…  │ fb      │               0 │         7101072165 │ … │      3.14198 │     0.603352 │        0 │
│ sponge/merge_ssd/b…  │ fb      │              

In [28]:
def print_for_dataset(dataset, algos, ratio):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT threads, ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND ratio={ratio}"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo, threads"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='threads', values='v', columns='algo')
    print(rows.to_markdown())
    rows[algos].to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/merge_thread/'+str(ratio)+'.txt')



algos = ['standard_merge', 'learned_merge_flatpgm256',  'learned_merge_btree256']
print_for_dataset("fb", algos, 1)
print_for_dataset("fb", algos, 10)
print_for_dataset("fb", algos, 100)
print_for_dataset("fb", algos, 1000)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)



## fb
|   threads |   hash_join |   inlj_btree256 |   lsj_sampledflatpgm256 |   sort_join |
|----------:|------------:|----------------:|------------------------:|------------:|
|         1 |     80.9313 |         49.3887 |                29.7188  |     31.2993 |
|         2 |     56.4348 |         37.4486 |                15.2571  |     29.4781 |
|         4 |     37.0868 |         19.6677 |                 9.97797 |     22.5739 |
|         6 |     31.0395 |         27.8541 |                 8.46116 |     20.3859 |
|         8 |     27.7622 |         24.0784 |                 7.71767 |     23.7554 |
## fb
|   threads |   hash_join |   inlj_btree256 |   lsj_sampledflatpgm256 |   sort_join |
|----------:|------------:|----------------:|------------------------:|------------:|
|         1 |    55.2928  |         8.00319 |                 7.40658 |     8.7249  |
|         2 |    28.0727  |         7.32919 |                 4.05965 |     7.20279 |
|         4 |    14.295   |         6.8212

In [29]:
def print_eps_vs_duration(dataset, algos, ratio):
    print("## " + dataset)
    algo_filter = ''
    for algo in algos:
        algo_filter = algo_filter + f" OR algo='{algo}'"
    algo_filter = algo_filter[3:]
    rows = duckdb.sql(
        " SELECT threads, ratio, epsilon, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}' AND ratio={ratio}"
        f" AND ({algo_filter})"
        " GROUP BY dataset, threads, ratio, algo, threads, epsilon"
        " ORDER BY ratio, threads"
    ).df()
    #print(rows)
    rows = rows.pivot(index='epsilon', values='v', columns='threads')
    print(rows.to_markdown())
    rows.to_csv('/home/chesetti/Repos/LearnedJoinDiskExp/experiments/results/join_eps_vs_duration/'+str(ratio)+'.txt')



algos = ['lsj_sampledflatpgm256', 'lsj_sampledflatpgm1024', 'lsj_sampledflatpgm4096']
print_eps_vs_duration("fb", algos, 1)
print_eps_vs_duration("fb", algos, 10)
print_eps_vs_duration("fb", algos, 100)
print_eps_vs_duration("fb", algos, 1000)
#print_for_dataset("wiki", algos)
#print_for_dataset("osm", algos)
#print_for_dataset("books", algos)


## fb
|   epsilon |       1 |       2 |        4 |       6 |       8 |
|----------:|--------:|--------:|---------:|--------:|--------:|
|       256 | 29.7188 | 15.2571 |  9.97797 | 8.46116 | 7.71767 |
|      1024 | 29.0431 | 15.4888 | 10.1069  | 8.4592  | 7.75957 |
|      4096 | 31.7732 | 16.4201 | 10.2662  | 8.61596 | 8.40002 |
## fb
|   epsilon |       1 |       2 |       4 |       6 |       8 |
|----------:|--------:|--------:|--------:|--------:|--------:|
|       256 | 7.40658 | 4.05965 | 2.30021 | 1.74046 | 1.45558 |
|      1024 | 7.28708 | 4.06699 | 2.30881 | 1.73226 | 1.45775 |
|      4096 | 7.05804 | 4.07814 | 2.31354 | 1.74289 | 1.47431 |
## fb
|   epsilon |       1 |       2 |       4 |       6 |        8 |
|----------:|--------:|--------:|--------:|--------:|---------:|
|       256 | 6.43783 | 3.16801 | 1.61441 | 1.1313  | 0.87751  |
|      1024 | 6.24528 | 3.16459 | 1.61388 | 1.12712 | 0.878693 |
|      4096 | 6.26147 | 3.18716 | 1.62527 | 1.12645 | 0.89718  |
## fb
|   ep